<a href="https://colab.research.google.com/github/sseydaltin/makine/blob/main/veri_inceleme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Veri Yükleme ve İnceleme

In [ ]:
import pandas as pd

# Veri setini yükleme
df = pd.read_csv('/content/heart_disease_uci.csv')

# Veri setinin ilk birkaç satırını görüntüleme
print("İlk 5 Satır:\n", df.head())

# Veri setinin genel bilgilerini inceleme
print("\nVeri Seti Bilgisi:\n")
print(df.info())

# Temel istatistiksel bilgiler
print("\nİstatistiksel Özet:\n")
print(df.describe())

# Eksik değer kontrolü
print("\nEksik Değerler:\n")
print(df.isnull().sum())


İlk 5 Satır:
    id  age     sex    dataset               cp  trestbps   chol    fbs  \
0   1   63    Male  Cleveland   typical angina     145.0  233.0   True   
1   2   67    Male  Cleveland     asymptomatic     160.0  286.0  False   
2   3   67    Male  Cleveland     asymptomatic     120.0  229.0  False   
3   4   37    Male  Cleveland      non-anginal     130.0  250.0  False   
4   5   41  Female  Cleveland  atypical angina     130.0  204.0  False   

          restecg  thalch  exang  oldpeak        slope   ca  \
0  lv hypertrophy   150.0  False      2.3  downsloping  0.0   
1  lv hypertrophy   108.0   True      1.5         flat  3.0   
2  lv hypertrophy   129.0   True      2.6         flat  2.0   
3          normal   187.0  False      3.5  downsloping  0.0   
4  lv hypertrophy   172.0  False      1.4    upsloping  0.0   

                thal  num  
0       fixed defect    0  
1             normal    2  
2  reversable defect    1  
3             normal    0  
4             normal  

# 2. Kategorik ve Sayısal Sütunları Ayırma

In [ ]:
# Kategorik ve sayısal sütunları belirleme
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Sonuçları ekrana yazdırma
print("Kategorik Sütunlar:", list(categorical_columns))
print("Sayısal Sütunlar:", list(numeric_columns))


Kategorik Sütunlar: ['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
Sayısal Sütunlar: ['id', 'age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca', 'num']


# 3. Eksik Veri Analizi

In [ ]:
# Eksik değerleri kontrol etme
missing_values = df.isnull().sum()
print("Eksik Değerler:\n", missing_values[missing_values > 0])

# Eksik veri doldurma (sayısal değişkenler için medyan ile doldurma)
df.fillna(df.median(numeric_only=True), inplace=True)

# Kategorik sütunlardaki eksik değerleri belirleme
missing_categorical = df.select_dtypes(include=['object', 'category']).isnull().sum()
missing_categorical = missing_categorical[missing_categorical > 0] # Sadece eksik değer içeren sütunları seç

for col in missing_categorical.index:  # Sadece eksik verisi olan sütunlarda işlem yap
    df[col].fillna(df[col].mode()[0], inplace=True)


# Eksik değerleri tekrar kontrol etme
print("\nEksik Değerler (Doldurma Sonrası):\n", df.isnull().sum().sum())

Eksik Değerler:
 trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
dtype: int64

Eksik Değerler (Doldurma Sonrası):
 0


<ipython-input-6-ea858718d8fe>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
<ipython-input-6-ea858718d8fe>:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col].fillna(df[col].mode()[0], inplace=True)


# 4. Encoding İşlemleri

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Label Encoding uygulanacak sütunlar (sıralı kategoriler)
label_encoding_columns = ['slope']  # Örnek olarak 'slope' seçildi, ihtiyaca göre genişletilebilir

for col in label_encoding_columns:
    label_encoder = LabelEncoder()
    df[col] = label_encoder.fit_transform(df[col])

# One-Hot Encoding uygulanacak sütunlar (sıralı olmayan kategoriler)
# Check if the columns are in the DataFrame before encoding
one_hot_encoding_columns = ['dataset', 'cp', 'restecg']
one_hot_encoding_columns = [col for col in one_hot_encoding_columns if col in df.columns]
# If not empty, apply one-hot encoding
if one_hot_encoding_columns:
    df = pd.get_dummies(df, columns=one_hot_encoding_columns, drop_first=True)

# Sonucu kontrol etme
print("Veri setinin güncellenmiş hali:\n", df.head())

Veri setinin güncellenmiş hali:
    id  age     sex  trestbps   chol    fbs  thalch  exang  oldpeak  slope  \
0   1   63    Male     145.0  233.0   True   150.0  False      2.3      0   
1   2   67    Male     160.0  286.0  False   108.0   True      1.5      1   
2   3   67    Male     120.0  229.0  False   129.0   True      2.6      1   
3   4   37    Male     130.0  250.0  False   187.0  False      3.5      0   
4   5   41  Female     130.0  204.0  False   172.0  False      1.4      2   

   ...               thal num  dataset_Hungary  dataset_Switzerland  \
0  ...       fixed defect   0            False                False   
1  ...             normal   2            False                False   
2  ...  reversable defect   1            False                False   
3  ...             normal   0            False                False   
4  ...             normal   0            False                False   

   dataset_VA Long Beach  cp_atypical angina  cp_non-anginal  \
0            

# 5. Standartlaştırma İşlemi

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming 'cleaned_heart_disease.csv' is the output from the previous steps
df = pd.read_csv('/content/heart_disease_uci.csv') # Load the dataframe

# Hedef değişkeni belirleme
target_column = 'target'

# Sayısal sütunları hedef değişken hariç olacak şekilde belirleme
numeric_columns = [col for col in df.select_dtypes(include=['float64', 'int64']).columns if col != target_column]

# StandardScaler ile standartlaştırma
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Güncellenmiş veri setini görüntüleme
print("Standartlaştırılmış veri setinin ilk 5 satırı:\n", df.head())

Standartlaştırılmış veri setinin ilk 5 satırı:
          id       age     sex    dataset               cp  trestbps      chol  \
0 -1.730169  1.007386    Male  Cleveland   typical angina  0.675287  0.305908   
1 -1.726404  1.432034    Male  Cleveland     asymptomatic  1.462483  0.784599   
2 -1.722639  1.432034    Male  Cleveland     asymptomatic -0.636705  0.269780   
3 -1.718873 -1.752828    Male  Cleveland      non-anginal -0.111908  0.459450   
4 -1.715108 -1.328180  Female  Cleveland  atypical angina -0.111908  0.043982   

     fbs         restecg    thalch  exang   oldpeak        slope        ca  \
0   True  lv hypertrophy  0.480653  False  1.303159  downsloping -0.724064   
1  False  lv hypertrophy -1.140262   True  0.569611         flat  2.487454   
2  False  lv hypertrophy -0.329805   True  1.578239         flat  1.416948   
3  False          normal  1.908602  False  2.403480  downsloping -0.724064   
4  False  lv hypertrophy  1.329704  False  0.477918    upsloping -0.724064 

# 6.Verinin Son Halini csv olarak kaydedin

In [ ]:
# Veri setini 'cleaned_heart_disease.csv' adıyla kaydetme
df.to_csv('cleaned_heart_disease.csv', index=False)

print("Veri seti başarıyla kaydedildi!")


Veri seti başarıyla kaydedildi!
